In [47]:
from utils import read_json
from glob import glob

In [48]:
data = []
for json_path in glob('data/*.json'):
    data.append(read_json(json_path))

In [49]:
print("total instances: ", len(data))

total instances:  4664


In [50]:
print(data[0])

{'data': {'id': '2348_timeout_clix_2022-09-01_dz_bluu', 'labels': [{'anotation_id': 0, 'step_1': ['HIB'], 'target_1': ['판단 불가'], 'step_2': ['HIB'], 'target_2': ['판단 불가'], 'step_3': ['HIB'], 'target_3': ['Others in broadcast'], 'comment': None, 'streamer_knowledge': None, 'twitch_knowledge': None}, {'anotation_id': 1, 'step_1': ['Incivility'], 'target_1': None, 'step_2': ['HIB'], 'target_2': ['Others in broadcast'], 'step_3': ['HIB'], 'target_3': ['Others in broadcast'], 'comment': None, 'streamer_knowledge': None, 'twitch_knowledge': None}, {'anotation_id': 2, 'step_1': ['HIB'], 'target_1': ['판단 불가'], 'step_2': ['HIB'], 'target_2': ['Others in broadcast'], 'step_3': ['HIB'], 'target_3': ['Others in broadcast'], 'comment': None, 'streamer_knowledge': None, 'twitch_knowledge': None}]}}


In [59]:
new_data = []
for example in data:
    mark = True
    for annotator in example['data']['labels']:
        if 'data_error' in annotator['step_1'] or 'date_error' in annotator['step_2'] or 'data_error' in annotator['step_3']:
            mark = False
    if mark:
        new_data.append(example)
    
print(len(new_data))

4583


In [62]:
exact_count = {
    'step_1': 0,
    'step_2': 0,
    'step_3': 0
}
count = 0

for example in new_data:
    example_labels = {}
    for step, target in [('step_1', 'target_1'), ('step_2', 'target_2'), ('step_3', 'target_3')]:
        example_labels[step] = []
        
        for annotator in example['data']['labels']:
            ### Target Consideration
            if '판단불가' in annotator[step]:
                annotator[step] = ['Incivility']                
#             if 'HIB' in annotator[step]:
#                 target_aggregation = []
#                 for l, t in zip(annotator[step], annotator[target]):
#                     if l == 'HIB':
#                         target_aggregation.append(l + "_" + t)
#                     else:
#                         target_aggregation.append(l)
#                 example_labels[step].append(target_aggregation)
#             else:
#                 example_labels[step].append(annotator[step])
            example_labels[step].append(annotator[step])    
            ### Target Non-Consideration
            #example_labels[step].append(annotator[step])
            
        if example_labels[step][0] == example_labels[step][1] == example_labels[step][2]:
            if len(example_labels[step][0])>1:
                count+=1
            exact_count[step] += 1

for step in exact_count:
    print(step, exact_count[step])
print(count)

step_1 1820
step_2 1884
step_3 1864
154


### Label Consolidation - List Intersection

In [63]:
list_intersection = {
    'step_1': {},
    'step_2': {},
    'step_3': {}
}

count = 0

for example in new_data:
    example_labels = {}
    for step, target in [('step_1', 'target_1'), ('step_2', 'target_2'), ('step_3', 'target_3')]:
        example_labels[step] = []
        
        for annotator in example['data']['labels']:
            ### Target Consideration
            if '판단불가' in annotator[step]:
                annotator[step] = ['Incivility']
                
#             if 'HIB' in annotator[step]:
#                 target_aggregation = []
#                 for l, t in zip(annotator[step], annotator[target]):
#                     if l == 'HIB':
#                         target_aggregation.append(l + "_" + t)
#                     else:
#                         target_aggregation.append(l)
#                 example_labels[step].append(target_aggregation)
#             else:
#                 example_labels[step].append(annotator[step])
            example_labels[step].append(annotator[step])
            ### Target Non-Consideration
            #example_labels[step].append(annotator[step])
        if ['data_error'] in example_labels[step]:
            continue             
        if len(set.intersection(*map(set, example_labels[step]))) > 0:
            for inter_label in list(set.intersection(*map(set, example_labels[step]))):
                if inter_label not in list_intersection[step]:
                    list_intersection[step][inter_label] = 0
                list_intersection[step][inter_label] += 1
#         else:
#             if ['Incivility'] in example_labels[step]:
#                 print(example_labels[step])
#                 count += 1
            
for step in list_intersection:
    total = 0
    for key in list_intersection[step]:
        total += list_intersection[step][key]
    print(step, total)

print(count)

step_1 2480
step_2 3439
step_3 3447
0


### Label Consolidation - Majority Vote

In [80]:
majority_vote = {
    'step_1': {},
    'step_2': {},
    'step_3': {}
}

targets_hib = {
    'step_1': {},
    'step_2': {},
    'step_3': {}
}

from collections import Counter

for example in new_data:
    example_labels = {}
    label_counter = {}
    for step, target in [('step_1', 'target_1'), ('step_2', 'target_2'), ('step_3', 'target_3')]:
        example_labels[step] = []
        for annotator in example['data']['labels']:
            ### Target Consideration
            if '판단불가' in annotator[step]:
                annotator[step] = ['Incivility']
                
#             if 'HIB' in annotator[step]:
#                 target_aggregation = []
#                 for l, t in zip(annotator[step], annotator[target]):
#                     if l == 'HIB':
#                         target_aggregation.append(l + "_" + t)
#                     else:
#                         target_aggregation.append(l)
#                 example_labels[step].append(target_aggregation)
#             else:
#                 example_labels[step].append(annotator[step])
            example_labels[step].append(annotator[step])
        
        
        label_counter[step] = Counter([item for sublist in example_labels[step] for item in sublist])
        
        assert len(label_counter[step].most_common(1)) == 1
        
        majority_label, majority_count = label_counter[step].most_common(1)[0]
        if majority_count > 1:
            if majority_label not in majority_vote[step]:
                majority_vote[step][majority_label] = 0
            majority_vote[step][majority_label] += 1
            
            if majority_label == 'HIB':
                targets = []
                for annotator in example['data']['labels']:
                    if annotator[target] is not None:
                        targets.append(annotator[target][0])
                majority_target, target_count = Counter(targets).most_common(1)[0]
                
                if majority_target not in targets_hib[step]:
                    targets_hib[step][majority_target] = 0
                
                if target_count > 1:
                    targets_hib[step][majority_target] += 1
                else:
                    if 'disagreement' not in targets_hib[step]:
                        targets_hib[step]['disagreement'] = 0
                    targets_hib[step]['disagreement'] += 1
                

            
for step in majority_vote:
    total = 0
    for key in majority_vote[step]:
        total += majority_vote[step][key]
    print(step, total)
    
print(targets_hib)


step_1 4439
step_2 4482
step_3 4482
{'step_1': {'판단 불가': 545, 'disagreement': 270, 'Broadcaster': 62, 'Others in broadcast': 67, 'Others outside of bradcast': 3, '판단불가': 0}, 'step_2': {'Others in broadcast': 320, 'Broadcaster': 712, 'disagreement': 129, '판단 불가': 16, 'Others outside of bradcast': 13}, 'step_3': {'Others in broadcast': 403, 'Broadcaster': 744, 'disagreement': 76, '판단 불가': 3, 'Others outside of bradcast': 20}}


In [81]:
print(majority_vote)

{'step_1': {'HIB': 947, 'Advertisements': 206, 'Begging': 64, 'Incivility': 2008, 'Excessive & Repetitive': 781, 'Discrimination': 104, 'Self-Destructive': 14, 'Backseating & Tall order': 153, 'Mentioning other broadcasters': 32, 'NSFW': 38, 'Spoiler': 1, 'Controversial Topic': 26, 'Illegal': 7, 'Specific Language Only': 43, 'Privacy': 15}, 'step_2': {'HIB': 1190, 'Advertisements': 197, 'Begging': 61, 'Incivility': 901, 'Specific Language Only': 311, 'Controversial Topic': 38, 'Excessive & Repetitive': 970, 'Discrimination': 101, 'Self-Destructive': 13, 'Backseating & Tall order': 166, 'Mentioning other broadcasters': 476, 'NSFW': 38, 'Spoiler': 1, 'Privacy': 16, 'Illegal': 3}, 'step_3': {'HIB': 1246, 'Advertisements': 198, 'Begging': 61, 'Incivility': 850, 'Specific Language Only': 311, 'Controversial Topic': 37, 'Excessive & Repetitive': 960, 'Discrimination': 105, 'Self-Destructive': 13, 'Backseating & Tall order': 169, 'Mentioning other broadcasters': 474, 'NSFW': 38, 'Spoiler': 1,

In [24]:
print(majority_vote)

{'step_1': {'HIB_판단 불가': 526, 'Advertisements': 201, 'Begging': 64, 'Incivility': 2005, 'Excessive & Repetitive': 772, 'Discrimination': 87, 'Self-Destructive': 14, 'Backseating & Tall order': 152, 'data_error': 80, 'HIB_Others in broadcast': 64, 'HIB_Broadcaster': 53, 'Mentioning other broadcasters': 27, 'NSFW': 38, 'Spoiler': 1, 'HIB_Others outside of bradcast': 3, 'Specific Language Only': 42, 'Privacy': 14, 'Controversial Topic': 20, 'Illegal': 6}, 'step_2': {'HIB_Others in broadcast': 300, 'Advertisements': 189, 'Begging': 60, 'HIB_Broadcaster': 691, 'Incivility': 893, 'Specific Language Only': 302, 'Controversial Topic': 33, 'Excessive & Repetitive': 934, 'Discrimination': 77, 'Self-Destructive': 13, 'Backseating & Tall order': 170, 'Mentioning other broadcasters': 486, 'date_error': 80, 'NSFW': 38, 'Spoiler': 1, 'HIB_Others outside of bradcast': 11, 'Privacy': 15, 'Illegal': 3, 'HIB_판단 불가': 16}, 'step_3': {'HIB_Others in broadcast': 390, 'Advertisements': 190, 'Begging': 60, 'HI